In [81]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [82]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [83]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [84]:
def create_dataFrame(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [85]:
train = pd.DataFrame()
train['image'], train['label'] = create_dataFrame(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [86]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [87]:
test = pd.DataFrame()
test['image'], test['label'] = create_dataFrame(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [88]:
print(test)

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


from tqdm.notebook import tqdm

In [89]:
from tqdm.notebook import tqdm

In [90]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)  # Convert list to a NumPy array before reshaping
    features = features.reshape(len(features),48,48,1) #hwhere depth = 1 since we're dealing with 2d images
    return features

In [91]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [92]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [93]:
 x_train = train_features/255.0
x_test = test_features/255.0

In [94]:
from sklearn.preprocessing import LabelEncoder

In [95]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [96]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [97]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [98]:
model = Sequential()
# convulational layers
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation = 'softmax'))

C:\Users\Saksham\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [99]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x = x_train, y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.2329 - loss: 1.8409 - val_accuracy: 0.2583 - val_loss: 1.8096
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 306s 1s/step - accuracy: 0.2475 - loss: 1.8062 - val_accuracy: 0.2628 - val_loss: 1.7648
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.2727 - loss: 1.7486 - val_accuracy: 0.3271 - val_loss: 1.6522
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 304s 1s/step - accuracy: 0.3306 - loss: 1.6530 - val_accuracy: 0.4355 - val_loss: 1.4700
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.3978 - loss: 1.5447 - val_accuracy: 0.4357 - val_loss: 1.4286
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.4274 - loss: 1.4708 - val_accuracy: 0.4960 - val_loss: 1.3246
Epoch 7/100
  6/226 ━━━━━━━━━━━━━━━━━━━━ 5:06 1s/step - accuracy: 0.4957 - loss: 1.3886